# US-Ignite Fed_Notebook

# Data from 2009-2016

In [2]:
#import libraries from Carto.
from cartoframes.auth import set_default_credentials
from cartoframes.viz import Map, Layer
import cartoframes
cartoframes.__version__
#from cartoframes.viz.helpers import color_continuous_layer
import pandas as pd
import geopandas as gpd
import requests
import json
from autocensus import Query #Library from Socrata to gather ACS Data
from cartoframes.data.services import Geocoding
#import censusdata
#set_default_credentials('creds_usignite.json')
set_default_credentials(
    base_url='https://usignite-admin.carto.com/',
    api_key='81c922dc336bd82103d92a768816a4567c444909'
)

from cartoframes.data.observatory import Catalog, Variable, Geography, Dataset

In [3]:
import numpy as np

from linearmodels import PanelOLS
from linearmodels import RandomEffects
import ipywidgets as widgets
from ipywidgets import Output, Tab
from IPython.display import clear_output
from cartoframes.data.services import Isolines
from cartoframes.data.observatory import Variable
from cartoframes.data.observatory import Enrichment
from cartoframes.data.observatory import Catalog

# Data Visulization

In [4]:
df_final=pd.read_csv("~/Downloads/Final_merged.csv")

In [5]:
df_final=df_final.reset_index()
df_final=df_final.drop(columns=['index'])


In [6]:
df_final=df_final[df_final['year']==2012]

In [7]:
df_final.head()

,Unnamed: 0,geo_id,pop,centroid,year,geometry,edut,collegemale,collegefemale,households,income,gini,total_p,ratio_65,zipcode,geoid,NAICS2012,Empratio,estabratio
2,509,6073015502,291.0,POINT (-116.8304254797509 32.81131603277056),2012,MULTIPOLYGON (((-116.8583888559341 32.81723278...,1790.0,124.0,107.0,113.0,904.0,0.3790,2748,0.108806,91901,6073015502,0,0.293542,0.029354
9,510,6073015502,291.0,POINT (-116.8304254797509 32.81131603277056),2012,MULTIPOLYGON (((-116.8583888559341 32.81723278...,1790.0,124.0,107.0,113.0,904.0,0.3790,2748,0.108806,91901,6073015502,51,0.293542,0.029354
16,1783,6073021202,319.0,POINT (-116.641978140868 32.8284525792826),2012,MULTIPOLYGON (((-116.7511618358752 32.79765279...,2101.0,92.0,133.0,121.0,1076.0,0.4807,3156,0.119455,91901,6073021202,0,14.090020,1.409002
23,1784,6073021202,319.0,POINT (-116.641978140868 32.8284525792826),2012,MULTIPOLYGON (((-116.7511618358752 32.79765279...,2101.0,92.0,133.0,121.0,1076.0,0.4807,3156,0.119455,91901,6073021202,51,14.090020,1.409002
30,1787,6073021204,416.0,POINT (-116.7638240859188 32.81083847358348),2012,MULTIPOLYGON (((-116.828524849101 32.794331787...,3683.0,378.0,399.0,156.0,1760.0,0.3916,5239,0.147929,91901,6073021204,0,1.761252,0.176125


In [8]:
#df_final['year']=df_final['year'].apply(lambda x: x.strftime('%Y') )

In [9]:
from shapely import wkt

df_final['geometry'] = df_final['geometry'].apply(wkt.loads)

In [10]:
gdf = gpd.GeoDataFrame(df_final, geometry='geometry', crs={'init': 'epsg:4326'})

In [11]:
gdf.head()

,Unnamed: 0,geo_id,pop,centroid,year,geometry,edut,collegemale,collegefemale,households,income,gini,total_p,ratio_65,zipcode,geoid,NAICS2012,Empratio,estabratio
2,509,6073015502,291.0,POINT (-116.8304254797509 32.81131603277056),2012,"MULTIPOLYGON (((-116.85839 32.81723, -116.8547...",1790.0,124.0,107.0,113.0,904.0,0.3790,2748,0.108806,91901,6073015502,0,0.293542,0.029354
9,510,6073015502,291.0,POINT (-116.8304254797509 32.81131603277056),2012,"MULTIPOLYGON (((-116.85839 32.81723, -116.8547...",1790.0,124.0,107.0,113.0,904.0,0.3790,2748,0.108806,91901,6073015502,51,0.293542,0.029354
16,1783,6073021202,319.0,POINT (-116.641978140868 32.8284525792826),2012,"MULTIPOLYGON (((-116.75116 32.79765, -116.7489...",2101.0,92.0,133.0,121.0,1076.0,0.4807,3156,0.119455,91901,6073021202,0,14.090020,1.409002
23,1784,6073021202,319.0,POINT (-116.641978140868 32.8284525792826),2012,"MULTIPOLYGON (((-116.75116 32.79765, -116.7489...",2101.0,92.0,133.0,121.0,1076.0,0.4807,3156,0.119455,91901,6073021202,51,14.090020,1.409002
30,1787,6073021204,416.0,POINT (-116.7638240859188 32.81083847358348),2012,"MULTIPOLYGON (((-116.82852 32.79433, -116.8284...",3683.0,378.0,399.0,156.0,1760.0,0.3916,5239,0.147929,91901,6073021204,0,1.761252,0.176125


In [12]:
from cartoframes import read_carto, to_carto
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer, basemaps, popup_element, basic_style, color_category_style, color_bins_style, default_legend, color_bins_legend, category_widget, formula_widget,  histogram_widget

from cartoframes.data.services import Isolines

from cartoframes.data.observatory import Catalog, Dataset
from cartoframes.data.observatory import Enrichment
from cartoframes.viz import Layer, animation_style, time_series_widget



In [13]:
gdf.head()  pop edut households income gini ratio_65 Empratioe stabratio

SyntaxError: invalid syntax (<ipython-input-13-02bd93bca05f>, line 1)

In [ ]:
Layout([
    Map(Layer('drought_wk_1', color_bins_style('dm', method='equal', palette='pinkyl'))),
    Map(Layer('drought_wk_2', color_bins_style('dm', method='equal', palette='pinkyl'))),
    Map(Layer('drought_wk_3', color_bins_style('dm', method='equal', palette='pinkyl'))),
    Map(Layer('drought_wk_4', color_bins_style('dm', method='equal', palette='pinkyl'))),
], is_static=False, viewport={'zoom': 3, 'lat': 33.4706, 'lng': -98.3457})

In [15]:
from cartoframes.auth import set_default_credentials
from cartoframes.viz import Map, Layer, Layout, color_bins_style

In [16]:
Layout([Map(
    Layer(gdf,
        color_bins_style('estabratio'),
        legends=color_bins_legend(
            title="estabratio",
            description='by census tract',
            footer='Data: US Census'
        ),
   
        popup_click=[
            popup_element('estabratio', ' Income'),
            popup_element('estabratio', ' Income'),
            popup_element('total_p', 'Total Population')
        ]
    ),
  
    basemap=basemaps.darkmatter
    
),
    Map(
    Layer(gdf,
        color_bins_style('edut'),
        legends=color_bins_legend(
            title="education",
            description='by census tract',
            footer='Data: US Census'
        ),
   
        popup_click=[
            popup_element('edut', ' education'),
            popup_element('total_p', 'Total Population')
        ]
    ),
  
    basemap=basemaps.darkmatter
    
),
       
                
                
                ], is_static=False)




In [73]:
mapfed = Map(
    Layer(gdf,
        color_bins_style('income'),
        legends=color_bins_legend(
            title="Estabalishment Ratio",
            description='by census tract',
            footer='Data: US Census'
        ),
     popup_hover=[
        popup_element('estabratio', ' Estabalishment Ratio'),
    ],
        popup_click=[
            
            popup_element('income', ' Income'),
            popup_element('pop', 'Total Population')
        ]
         ,
    widgets=[
        formula_widget(
            ' pop',
          
            title='Pedestrians Involved'
        ),
        formula_widget(
            ' edut',
           
            title='Cyclists Involved'
        ),
        category_widget(
            'households',
            title='Collision Type'
        )
    ]
    
    ),
    
 
    basemap=basemaps.darkmatter

  
)
       
                
                 




In [67]:
from cartoframes.viz import Map, Layer, color_continuous_legend, color_continuous_style, formula_widget, time_series_widget, basemaps, basic_style


In [74]:
mapfed = Map ([
    Layer(
        gdf,
        style=color_continuous_style('estabratio', size=4, stroke_width=0, animate='estabratio'),
        legends=color_continuous_legend(
            title="Estabalishment Ratio",
            description='by census tract',
            footer='Data: US Census'
        ),
        widgets=[
            formula_widget('estabratio', operation='sum', title='estabratio'),
          
        ],
    ),
    Layer(
        gdf,
        color_bins_style('income'),
            popup_click=[popup_element('income', ' Estabalishment Ratio')]
    
    )],
    basemap=basemaps.darkmatter,
    show_info=True,
   
)

In [75]:
mapfed.publish(
    name='map sample',
    password='11111',
    if_exists='replace')

{'id': 'fa8c5fcb-b1fe-4cbc-9b24-9dccb86cba27',
 'url': 'https://usignite.carto.com/u/usignite-admin/kuviz/fa8c5fcb-b1fe-4cbc-9b24-9dccb86cba27',
 'name': 'map sample',
 'privacy': 'password'}

# Data Enrichment

In [124]:
widgets.IntSlider()


IntSlider(value=0)

In [125]:
def gpd_bbox(bounds):
    '''
    bounds (list): min_lng, min_lat, max_lng, max_lat
    '''
    from shapely.geometry import Polygon
    import geopandas as gpd
    import pandas as pd
    bbox = gpd.GeoDataFrame(pd.DataFrame([Polygon.from_bounds(*bounds)], columns=['geometry']))
    return bbox

def tile_cover(geometry, z):
    import geopandas as gpd
    from supermercado import burntiles, super_utils
    from pygeotile.tile import Tile
    geo = gpd.GeoSeries([geometry]).__geo_interface__['features'][0]
    geo = [f for f in super_utils.filter_polygons([geo])]
    return [Tile.from_google(*geo).quad_tree for geo in [f for f in burntiles.burn(geo, z)]]

def quad_to_poly(quad):
    from shapely.geometry import Polygon
    from pygeotile.tile import Tile
    from pygeotile import tile
    bbox = tile.Tile.from_quad_tree(quad).bounds
    # min_lng, min_lat, max_lng, max_lat
    poly = Polygon.from_bounds(bbox[0][1], bbox[0][0], bbox[1][1], bbox[1][0]) 
    return poly

def gpd_grid(geometry, z, tag=None):
    '''
    draw grid and return gpd.GeoDataFrame()
    '''
    from shapely.geometry import Polygon, MultiPolygon
    import pandas as pd
    import geopandas as gpd
    
    if isinstance(geometry, Polygon) or isinstance(geometry, MultiPolygon):
        qt_list = tile_cover(geometry, z)
    elif isinstance(geometry, list) and len(geometry)==4:
        qt_list = tile_cover(gpd_bbox(geometry).geometry.values[0], z)
    else:
        raise TypeError('''
        geometry should be either 
        shapely.Polygon (or shapely.MultiPolygon) 
        or list (min_lng, min_lat, max_lng, max_lat)')
        ''')
    grid = pd.DataFrame(qt_list, columns=['qt'])
    grid['geometry'] = grid['qt'].apply(lambda x: quad_to_poly(x))
    grid['tag'] = [tag] * len(grid)
    return gpd.GeoDataFrame(grid)

def qt_convert(lat, lng, zoom=19):
    '''
    lat, lng, zoom=18(default)
    '''
    from pygeotile.tile import Tile
    try:
        return Tile.for_latitude_longitude(lat, lng, zoom=zoom).quad_tree
    except:
        pass
    
# def run_api(ngrok_address, api_call="add/1/2", pause=1, timeout=600):
#     import time
#     import requests
#     exec_call = ngrok_address + api_call
#     cid = requests.get(exec_call).text.split('>')[1].split('<')[0].split('check status of')[1].strip()
#     output_call = ngrok_address + f"check/{cid}?external=True"
    
#     result = requests.get(output_call)
#     start_time = time.time()
#     process_time = float('inf')
#     while result.text=='PENDING' :
#         time.sleep(pause)
#         result = requests.get(output_call)
#         process_time = time.time() - start_time
#         print(process_time)
#         if process_time>timeout:
#             break
#     return result

def run_api(ngrok_address, api_call="add/1/2", pause=1, timeout=600):
    import time
    import requests
    exec_call = ngrok_address + api_call
    cid = requests.get(exec_call).text.split('>')[1].split('<')[0].split('check status of')[1].strip()
    output_call = ngrok_address + f"check/{cid}?external=True"
    
    result = requests.get(output_call)
    start_time = time.time()
    process_time = float('inf')
    while result.text=='PENDING' :
        time.sleep(pause)
        result = requests.get(output_call)
        process_time = time.time() - start_time
        print(process_time)
        if process_time>timeout:
            break
    return result



In [126]:
auth_username = widgets.Text(
                    placeholder='Username',
                    description='Username:',
                    disabled=False)
auth_apikey = widgets.Text(
                    placeholder='API Key',
                    description='API Key:',
                    disabled=False)
auth_confirm = widgets.Button(description="Authentication",
                              layout=widgets.Layout(width='25%'))
auth_confirm.style.button_color = 'lightgreen'

@auth_confirm.on_click
def auth(*ignore):
    auth_apikey.disabled=True
    auth_username.disabled=True
    set_default_credentials(username=auth_username.value,
                            api_key=auth_apikey.value)
    

################################################################################################################################

data_tablename = widgets.Text(
                    placeholder='Table Name',
                    description='Table Name:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))

data_confirm = widgets.Button(description="Confirm",
                              layout=widgets.Layout(width='25%'))
data_confirm.style.button_color = 'lightgreen'

data_preview = widgets.Button(description="Preview",
                              layout=widgets.Layout(width='25%'))
data_preview.style.button_color = 'lightblue'
data_preview_map = widgets.HTML()

@data_preview.on_click
def preview(*ignore):
    preview_map = Map(Layer(data_tablename.value), show_info=True)
    data_preview_map.value = preview_map._repr_html_()
    
@data_confirm.on_click
def tablename_confirm(*ignore):
    data_tablename.disabled=True
    
################################################################################################################################

arrow = widgets.HTML("""<font size="+1000">&#10144;</font>""")

enrich_step_origin = widgets.ToggleButton(
    value=True,
    description='Origin',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

enrich_step_isochrone = widgets.ToggleButton(
    value=True,
    description='Isochrone',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)


enrich_step_run = widgets.Button(
    value=True,
    description='Enrich',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

################################################################################################################################

@enrich_step_origin.observe
def step_origin_check(*ignore):
    if enrich_step_origin.value==True:
        enrich_step_origin.icon = 'check'
    elif enrich_step_origin.value==False:
        enrich_step_origin.icon = ''
        
@enrich_step_isochrone.observe
def step_isochrone_check(*ignore):
    if enrich_step_isochrone.value==True:
        enrich_step_isochrone.icon = 'check'
    elif enrich_step_isochrone.value==False:
        enrich_step_isochrone.icon = ''

        
############################################################################
origin_tablename = widgets.Text(
                    placeholder='Table Name',
                    description='Table Name:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))

origin_var = widgets.Text(value='lat,lng',
                    placeholder='Y, lat, lng, etc ... (e.g. rf,lat,lng )',
                    description='Variable:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))
############################################################################

iso_mode = widgets.Dropdown(options=['car', 'walk'],
                            disabled=False,
                            value='car',
                            description='Mode: ',
                            layout=widgets.Layout(height='auto', width='30%'))

iso_ranges = widgets.Text(value='',
                          placeholder='Travel Time Values in Seconds (e.g. 60,180,240, ...)',
                          description='Ranges: ',
                          layout=widgets.Layout(height='auto', width='60%'),
                          disabled=False)
############################################################################

default_provider = 'usa_acs'
default_datasets = list(Catalog().provider(provider_id=default_provider).datasets.to_dataframe()['id'].values)

enrich_feature_providers = widgets.Tab(layout=widgets.Layout(width='250px', height='250px'))
enrich_feature_providers.set_title(0, 'Provider')
enrich_feature_providers.children = [widgets.Select(rows=10, 
                                                    value=default_provider,
                                                    options=['usa_acs', 'ags'], 
                                                    layout=widgets.Layout(width='100%'))] 

enrich_feature_datasets = widgets.Tab(layout=widgets.Layout(width='300px', height='250px'))
enrich_feature_datasets.set_title(0, 'Dataset')
enrich_feature_datasets.children = [widgets.Select(rows=10, 
                                                   value='',
                                                   options=default_datasets+[''],
                                                   layout=widgets.Layout(width='100%'))] 

enrich_feature_features = widgets.Tab(layout=widgets.Layout(width='300px', height='250px'))
enrich_feature_features.set_title(0, 'Feature')
enrich_feature_features.children = [widgets.SelectMultiple(rows=10, layout=widgets.Layout(width='100%'))] 
enrich_feature_features.children[0].options = []

enrich_feature_selected = widgets.Tab(layout=widgets.Layout(width='300px', height='250px'))
enrich_feature_selected.set_title(0, 'Selected [0]')
enrich_feature_selected.children = [widgets.SelectMultiple(rows=10, layout=widgets.Layout(width='100%'))] 
enrich_feature_selected.children[0].options = []

def dataset_update(*ignore):
    provider = enrich_feature_providers.children[0].value
    enrich_feature_datasets.children[0].options = list(Catalog().provider(provider_id=provider).datasets.to_dataframe()['id'].values) + ['']
    enrich_feature_datasets.children[0].value = ''

def feature_update(*ignore):
    if enrich_feature_datasets.children[0].value != '':
        dataset = Dataset.get(enrich_feature_datasets.children[0].value)
        enrich_feature_features.children[0].options = dataset.variables.to_dataframe()['id'].values
    else:
        enrich_feature_features.children[0].options = []

enrich_feature_providers.children[0].observe(dataset_update, ['value', 'options'])
enrich_feature_datasets.children[0].observe(feature_update, ['value', 'options'])


add_feature = widgets.Button(description="+", layout=widgets.Layout(width='80%', justify_content='center'))
remove_feature = widgets.Button(description="-", layout=widgets.Layout(width='80%', justify_content='center'))

@add_feature.on_click
def add_selected_feature(*ignore):
    if len(enrich_feature_features.children[0].value) != 0:
        enrich_feature_selected.children[0].options = list(set(list(enrich_feature_selected.children[0].options) + \
                                                               list(enrich_feature_features.children[0].value)))
        enrich_feature_selected.set_title(0, f'Selected [{len(enrich_feature_selected.children[0].options)}]')
        
@remove_feature.on_click
def remove_selected_feature(*ignore):
    if len(enrich_feature_selected.children[0].value) != 0:
        all_selected = list(enrich_feature_selected.children[0].options)
        for _ in enrich_feature_selected.children[0].value:
            all_selected.remove(_)
        enrich_feature_selected.children[0].options = all_selected
        enrich_feature_selected.set_title(0, f'Selected [{len(enrich_feature_selected.children[0].options)}]')
        

feature_info_board = widgets.HTML(disabled=True,
                                  layout=widgets.Layout(width='99%', 
                                                        height='auto',
                                                        display='flex',
                                                        flex_flow='row',
                                                        justify_content='center'))

def update_feature_info(*ignore):
    if enrich_feature_datasets.children[0].value != '':
        feature_info_board.value = f"<b>[Dataset]</b>: {Dataset.get(enrich_feature_datasets.children[0].value).to_dict()['name']}"
        feature_info_board.value += '<br/>'
        if len(enrich_feature_features.children[0].value) != 0:
            feature_info_board.value += f"<b>[Feature]</b>: {Variable.get(enrich_feature_features.children[0].value[0]).to_dict()['description']}"
            feature_info_board.value += '<br/>'
    else:
        feature_info_board.value = ''


enrich_feature_providers.children[0].observe(update_feature_info, ['value', 'options'])
enrich_feature_datasets.children[0].observe(update_feature_info, ['value', 'options'])
enrich_feature_features.children[0].observe(update_feature_info, ['value', 'options'])

############################################################################

enrich_accordion = widgets.Accordion(children=[widgets.HBox([origin_tablename, origin_var]),
                                               widgets.HBox([iso_mode, iso_ranges]),
                                               widgets.Text(value='tmp_iso',
                                                            placeholder='Table Name',
                                                            description='Table Name:',
                                                            disabled=False,
                                                            layout=widgets.Layout(width='50%')),
                                               widgets.VBox([widgets.HBox([enrich_feature_providers, 
                                                                           enrich_feature_datasets, 
                                                                           enrich_feature_features, 
                                                                           widgets.VBox([add_feature, remove_feature],
                                                                                         layout=widgets.Layout(width='50px', 
                                                                                                               display='flex',
                                                                                                               height='auto', 
                                                                                                               justify_content='center')),
                                                                           enrich_feature_selected]),
                                                             feature_info_board]),
                                               widgets.VBox([widgets.Text(value='tmp_origin',
                                                                         placeholder='Table Name',
                                                                         description='Origin:',
                                                                         disabled=False,
                                                                         layout=widgets.Layout(display='flex', width='auto'))
                                                            ])
                                              ])

enrich_accordion.set_title(0, 'Tables')
enrich_accordion.set_title(1, 'Isochrone Setting')
enrich_accordion.set_title(2, 'Isochrone Save')
enrich_accordion.set_title(3, 'Enrichment Features')
enrich_accordion.set_title(4, 'Enrichment Save')


@enrich_step_run.on_click
def run_enrichment(*ignore):
    if (enrich_step_origin.value == True) and (enrich_accordion.children[-1].children[0].value != ''):
        table_name = origin_tablename.value
        locations = CartoDataFrame.from_carto(table_name)
    
        if (enrich_step_isochrone.value == True) and (iso_ranges.value != '') and (enrich_accordion.children[2].value != ''):
            mode = iso_mode.value
            ranges = [float(_.strip()) for _ in enrich_accordion.children[1].children[1].value.split(',')]
            if len(ranges) == 1:
                locations_isochrones, _ = Isolines().isochrones(locations, ranges=ranges, mode=mode)
                locations_isochrones.to_carto(table_name=enrich_accordion.children[2].value, if_exists='replace')

        features = list(enrich_feature_selected.children[0].options)
        if len(features) != 0:
            origin = locations.copy()
            origin['geometry'] = origin['the_geom']
            if enrich_step_isochrone.value == True:
                iso = CartoDataFrame.from_carto(enrich_accordion.children[2].value)
                origin['geometry'] = iso['the_geom']

            enrichment_origin = origin[[_ for _ in ['geometry']+[_.strip() for _ in origin_var.value.split(',') if _!='']]]
            enrichment_origin = Enrichment().enrich_polygons(enrichment_origin, features)
            enrichment_origin.to_carto(table_name=enrich_accordion.children[-1].children[0].value, if_exists='replace')
    

In [127]:
accordion = widgets.Accordion(children=[widgets.VBox([auth_username, widgets.HBox([auth_apikey, widgets.HTML('<span style="padding-left:100px"></span>'), auth_confirm])]),
                                        widgets.VBox([widgets.HBox([data_tablename, data_preview, data_confirm]),
                                                      data_preview_map]),
                                        widgets.VBox([widgets.HBox([enrich_step_origin, arrow,
                                                                    enrich_step_isochrone, arrow,
                                                                    enrich_step_run], 
                                                      layout=widgets.Layout(height='auto', 
                                                                            flex_flow='row',
                                                                            justify_content='center')),
                                                      enrich_accordion
                                                     ]),
                            
                      
                                       ])

In [128]:
accordion.set_title(0, 'Carto Account')
accordion.set_title(1, 'Data')
accordion.set_title(2, 'Enrichment')

accordion

Accordion(children=(VBox(children=(Text(value='', description='Username:', placeholder='Username'), HBox(child…